In [57]:
spark.sql(
    """
   SHOW tables from PL7G1_GOLD
   """
).toPandas()


namespace                   tableName  isTemporary
0   pl7g1_gold           air_quality_gases        False
1   pl7g1_gold    air_quality_gases_presto        False
2   pl7g1_gold               incendios_ano        False
3   pl7g1_gold        incendios_ano_presto        False
4   pl7g1_gold         incendios_bombeiros        False
5   pl7g1_gold  incendios_bombeiros_presto        False
6   pl7g1_gold             incendios_covid        False
7   pl7g1_gold      incendios_covid_presto        False
8   pl7g1_gold        incendios_mes_presto        False
9   pl7g1_gold            incendios_semana        False
10  pl7g1_gold     incendios_semana_presto        False

In [34]:
spark.sql(
    """
    DROP TABLE IF EXISTS PL7G1_GOLD.Air_Quality_Incendios
    """
)

DataFrame[]

In [59]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [60]:
air_quality = spark.table("PL7G1_GOLD.Air_Quality_Gases")
incendios = spark.table("PL7G1_GOLD.Incendios_Ano")

In [61]:
incendios_Ar = incendios.join(air_quality, (incendios.case_year == air_quality.Year) & (incendios.borough == air_quality.Borough),"inner").drop(incendios.borough)

In [62]:
incendios_Ar =incendios_Ar.drop(air_quality.Year)

In [63]:
incendios_Ar.printSchema()

root
 |-- case_year: integer (nullable = true)
 |-- Appliances: long (nullable = true)
 |-- Cooking_Carelessness: long (nullable = true)
 |-- Electrical: long (nullable = true)
 |-- Electrical_Devices: long (nullable = true)
 |-- Heaters: long (nullable = true)
 |-- Hot_Objects: long (nullable = true)
 |-- Ignitable: long (nullable = true)
 |-- Incendiary: long (nullable = true)
 |-- Motors: long (nullable = true)
 |-- Natural_Sources: long (nullable = true)
 |-- No_Fire: long (nullable = true)
 |-- Open_Flame: long (nullable = true)
 |-- Other: long (nullable = true)
 |-- Sem_Informacao: long (nullable = true)
 |-- Smoking: long (nullable = true)
 |-- Total_Fires: long (nullable = true)
 |-- Borough: string (nullable = true)
 |-- PM2_5_SUM: double (nullable = true)
 |-- PM2_5_AVG: double (nullable = true)
 |-- PM2_5_MIN: double (nullable = true)
 |-- PM2_5_MAX: double (nullable = true)
 |-- NO2_SUM: double (nullable = true)
 |-- NO2_AVG: double (nullable = true)
 |-- NO2_MIN: double (

In [64]:
spark.sql(
    """
    CREATE EXTERNAL TABLE PL7G1_GOLD.Incendios_Air_Quality (
        case_year int,
        Borough VARCHAR(50),
        PM2_5_SUM Double,
        PM2_5_AVG Double,
        PM2_5_MIN Double,
        PM2_5_MAX Double,
        NO2_SUM Double,
        NO2_AVG Double,
        NO2_MIN Double,
        NO2_MAX Double,
        O3_SUM Double,
        O3_AVG Double,
        O3_MIN Double,
        O3_MAX Double,
        SO2_SUM Double,
        SO2_AVG Double,
        SO2_MIN Double,
        SO2_MAX Double,
        Appliances long,
        Cooking_Carelessness long,
        Electrical long, 
        Electrical_Devices long,
        Heaters long,
        Hot_Objects long,
        Ignitable long,
        Icendiary long,
        Motors long,
        Natural_Sources long,
        No_Fire long,
        Open_Flame long,
        Other long,
        Sem_Informacao long,
        Smoking long,
        Total_Fires long
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/Incendios_Air_Quality'
    """
)

DataFrame[]

In [65]:
incendios_Ar \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true")\
    .save("hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/Incendios_Air_Quality")

In [66]:
spark.table("PL7G1_GOLD.Incendios_Air_Quality").show()

+---------+-------------+------------------+-----------------+---------+---------+------------------+------------------+-------+-------+------------------+------------------+------+------+-------+-------+-------+-------+----------+--------------------+----------+------------------+-------+-----------+---------+---------+------+---------------+-------+----------+-----+--------------+-------+-----------+----------+
|case_year|      Borough|         PM2_5_SUM|        PM2_5_AVG|PM2_5_MIN|PM2_5_MAX|           NO2_SUM|           NO2_AVG|NO2_MIN|NO2_MAX|            O3_SUM|            O3_AVG|O3_MIN|O3_MAX|SO2_SUM|SO2_AVG|SO2_MIN|SO2_MAX|Appliances|Cooking_Carelessness|Electrical|Electrical_Devices|Heaters|Hot_Objects|Ignitable|Icendiary|Motors|Natural_Sources|No_Fire|Open_Flame|Other|Sem_Informacao|Smoking|Total_Fires|Incendiary|
+---------+-------------+------------------+-----------------+---------+---------+------------------+------------------+-------+-------+------------------+-----------

In [71]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/Incendios_Air_Quality/`
""").show()

++
||
++
++



In [72]:
spark.sql("""
    CREATE EXTERNAL TABLE IF NOT EXISTS PL7G1_GOLD.Incendios_Air_Quality_Presto (
        case_year int,
        Borough VARCHAR(50),
        PM2_5_SUM Double,
        PM2_5_AVG Double,
        PM2_5_MIN Double,
        PM2_5_MAX Double,
        NO2_SUM Double,
        NO2_AVG Double,
        NO2_MIN Double,
        NO2_MAX Double,
        O3_SUM Double,
        O3_AVG Double,
        O3_MIN Double,
        O3_MAX Double,
        SO2_SUM Double,
        SO2_AVG Double,
        SO2_MIN Double,
        SO2_MAX Double,
        Appliances long,
        Cooking_Carelessness long,
        Electrical long, 
        Electrical_Devices long,
        Heaters long,
        Hot_Objects long,
        Ignitable long,
        Icendiary long,
        Motors long,
        Natural_Sources long,
        No_Fire long,
        Open_Flame long,
        Other long,
        Sem_Informacao long,
        Smoking long,
        Total_Fires long
    )
    ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
    STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
    OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
    LOCATION 'hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/Incendios_Air_Quality/_symlink_format_manifest/'
    """).show()

++
||
++
++



In [ ]:

#.select("Year", "Borough", "PM2_5_SUM", "PM2_5_AVG", "PM2_5_MIN", "PM2_5_MAX", "SUM_PM2_5", "AVG_PM2_5", "MAX_PM2_5", "MIN_PM2_5", "SUM_SO2", "AVG_SO2", "MAX_SO2", "MIN_SO2", "SO2_SUM", "SO2_AVG", "SO2_MIN", "SO2_MAX")